In [5]:
import re
# pid = !netstat -a -v | grep 8050
# pid = int(re.split('\s+', pid[0])[-4])
# !kill {pid}

import pandas as pd

import jupyter_dash
from dash import html
import dash
import dash_bootstrap_components as dbc
from dash import dcc
from dash.dependencies import Input, Output, State

from IPython.lib import backgroundjobs as bg
from IPython.display import IFrame as WebBrowser
import plotly.express as px

import calendar

jobs = bg.BackgroundJobManager()
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])


width = 600
height = int(1.6180339887 * width)

style_header = {"text-align": "center"}

mimic_url = 'https://mimic.mit.edu/docs/iii/'
dash_url = 'https://dash.plotly.com/'

style_main_adaptive = { 'lg':{'size':6, 'offset':2}, 'md':{'size':12, 'offset':2}  } 

app_title = html.H1('Welcome to the DASHboard of Alberto and Linda', style = style_header)
app_title_dash1 = html.H2('Dashboard 1', style = style_header)
app_title_dash2 = html.H2('Dashboard 2', style = style_header)
app_title_dash3 = html.H2('Dashboard 3', style = style_header)

app_items = html.Ul([ 
                html.Li(['Source: ', html.A(mimic_url, href=mimic_url)]),
                html.Li(['Made with : ', html.A('Plotly Dash', href=dash_url)]), 
            ]
        )


# mimic_tables = [ 'calories' ]

# mimic_tables_attributes = {
# 'calories': re.split('\s+','''value float 
#             vt datetime
#             partecipant varchar(3)''')

# }



from pony.orm import Database, Required, Optional, PrimaryKey, Set, select, count, db_session
from  datetime import datetime,date
import psycopg2

#Connessione al DB !RIcordarsi di far andare il server!
DB = Database()
DB.bind(provider='postgres', 
        database='postgres',
        user='postgres', 
        port='5433',  #Linda: 5432
        host='localhost',
        password='admin'
        )

class Partecipant(DB.Entity):
    _table_  = 'partecipant'
    name = PrimaryKey(str)
    age  = Optional(int)
    height = Optional(int)
    gender = Optional(str)
    max_heart_rate = Optional(int)
    group = Optional(str)
    base_line_cal = Optional(float)
    calories = Set('Calories')
    BPM = Set('BPM')
    exercises = Set('Exercises')
    sleep = Set('Sleep')
    day = Set('Day')

class Calories(DB.Entity):
    _table_ = 'calories'
    value = Required(float)
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)
    
class Admission(DB.Entity):
    _table_ = "admissions"
    row_id = PrimaryKey(int)
    vts = Optional(datetime, column="admittime")
    adm_type = Optional(str, column="admission_type")
    location = Optional(str, column="admission_location")
    patient = Optional('Patient', column='subject_id')

class BPM(DB.Entity):
    _table_ = 'bpm'
    vt = Required(datetime)
    value = Optional(int)
    confidence = Optional(int)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)

class Exercises(DB.Entity):
    _table_ = "exercises"
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    # attributes of the json
    activityName = Required(str)
    avg_BPM = Optional(float)
    calories = Optional(int)
    duration = Optional(int)
    steps = Optional(float)
    elevationGain = Optional(float)
    PrimaryKey(vt, partecipant)

class Sleep(DB.Entity):
    _table_ ="sleep"
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    # attributes of the json
    duration = Optional(float)
    endTime = Optional(datetime)
    minutesAsleep = Optional(int)
    types = Optional(str)
    efficiency = Optional(int)
    PrimaryKey(partecipant, vt)

class Day(DB.Entity):
    _tabel_ = "day"
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)
    n_lightly_active_minutes = Optional(int)
    n_moderately_active_minutes = Optional(int)
    f_resting_heart_rate = Optional(float)
    n_sedentary_minutes = Optional(int)
    n_very_active_minutes = Optional(int)
    n_overall_score = Optional(int)
    n_composition_score = Optional(int)
    n_revitalization_score = Optional(int)
    n_duration_score = Optional(int)
    n_deep_sleep_in_minutes = Optional(int)
    f_time_in_heart_rate_BELOW_DEFAULT_ZONE_1 = Optional(float)
    f_time_in_heart_rate_zone_1 = Optional(float)
    f_time_in_heart_rate_zone_2 = Optional(float)
    f_time_in_heart_rate_zone_3 = Optional(float)
    
class Patient(DB.Entity):
    _table_ = "patients"
    row_id = PrimaryKey(int)
    admission = Set(Admission)
    gender = Optional(str)
    
class Prescription(DB.Entity):
    _table_ = "prescriptions"
    row_id = PrimaryKey(int)    


DB.generate_mapping()

def extract_entity_name(entity):
    return re.split('\.|"|\'', str(entity))[-2]
    



count(r for r in eval(extract_entity_name(Calories)))

str(datetime.now())



# @db_session
# def entity_display(entity_name):
#        return f'at {str(datetime.now())[:-7]} the number of instances in {entity_name} is  {count(r for r in eval(entity_name))}'

# entity_display('Calories')

# jobs = bg.BackgroundJobManager()
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# @app.callback(Output('selected_entity_output', 'children'), Input('selected_entity', 'value'))
# def display_selected_entity(entity_name):
#     if entity_name is None:
#         r = 'No Entity selected'
#     else:
#         try: 
#              r = entity_display(entity_name)
#         except Exception as e:
#              r = str(e)
#     return r    

In [6]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from pony.orm import db_session, select
from flask import Flask

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])




In [7]:
import plotly.express as px
@db_session
def scatter_by_month(year = None, month = None):
    data = list(select( 
        ((a.vt, a.vt.date().year, a.vt.date().month, a.activityName, a.avg_BPM, a.calories, c.name, c.age) for a in Exercises
        for c in Partecipant 
        if a.partecipant == c
        )
        ))
    data.sort(key=lambda x: [x[0]])   
    data
    df = pd.DataFrame(data, columns=['Data', 'Year', 'Month','Activity name','Bpm','Cal','Partecipant','Age'])
    df = df[ df["Year"] == year] if year != None else df
    df = df[ df["Month"] == month] if month != None else df
    print(df)

    fig = px.scatter(
        data_frame = df,
        x = "Data",
        y= "Bpm",
        size= "Cal",
        color = 'Partecipant',
        # animation_frame= "Age",
        
        # facet_col= "Gender"
    )

    fig2 = px.line(df, x='Data', y='Bpm', color='Partecipant', markers=True)
    fig2.update_layout(showlegend = False)

    fig2.for_each_trace(
        lambda trace:
        trace.update(showlegend = False)
    )

    # fig.add_trace(fig2.data[0], )
    for t in fig2.data:
        fig.add_trace(t)

    # fig.show()
    return fig

scatter_by_month(2019,11)

                 Data  Year  Month Activity name    Bpm   Cal Partecipant  Age
0 2019-11-07 09:04:11  2019     11          Walk  104.0   202         p07   26
1 2019-11-07 16:08:50  2019     11       Workout  134.0   746         p07   26
2 2019-11-09 09:21:55  2019     11       Workout  144.0  1543         p07   26
3 2019-11-09 12:07:33  2019     11       Workout  137.0  1568         p07   26


In [8]:
@db_session
def get_available_years():
    # Query per ottenere gli anni disponibili dal database utilizzando Pony ORM
    years = list(select((y.vt.date().year) for y in Exercises))
    return years

@db_session
def get_available_months(year):
    # Query per ottenere i mesi disponibili dal database per l'anno selezionato utilizzando Pony ORM
    months = list(select(m.vt.date().month for m in Exercises if m.vt.date().year == year))
    return months


app.layout = html.Div([
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': str(year), 'value': year} for year in get_available_years()],
        placeholder='Select a year...'
    ),
    html.Div(id='month-buttons'),
    html.Div(id='selected-info')  # Div per visualizzare l'anno e il mese selezionati
    # dcc.Graph(id='scatter-plot')
])

@app.callback(
    Output('month-buttons', 'children'),
    [Input('year-dropdown', 'value')]
)
def update_month_buttons(year):
    if year is None:
        return []

    months = get_available_months(year)

    buttons = [
        dbc.Button(f'{calendar.month_name[month]}', id=f'month-buttons-{month}',className="mx-3 mt-3", n_clicks=0 )
        for month in months
    ]

    return buttons

@app.callback(
    Output('selected-info', 'children'),
    [Input(f'month-button-{month}', 'n_clicks') for month in range(1, 13)],
    [State('year-dropdown', 'value')]
)
def update_selected_info(*args):
    month_clicked = [month for month, n_clicks in enumerate(args, start=1) if n_clicks > 0]
    
    if len(month_clicked) == 0:
        return []
    print(month_clicked)
    year = app.callback_context.states['year-dropdown.value']
    month = month_clicked[0]

    return f"Anno selezionato: {year}, Mese selezionato: {month}"


@app.callback(
    Output('scatter-plot', 'figure'),
    [Input(f'month-button-{month}', 'n_clicks') for month in range(1, 13)]
)
def update_scatter_plot(*args):
    month_clicked = [month for month, n_clicks in enumerate(args, start=1) if n_clicks > 0]
    if len(month_clicked) == 0:
        return {}

    month_clicked = [month for month, n_clicks in enumerate(args, start=1) if n_clicks > 0]
    if len(month_clicked) == 0:
        return scatter_by_month()
    
    year = app.callback_context.inputs['year-dropdown.value']
    month = month_clicked[0]
    print(year, month)

    if year != None and month != None:
        fig = scatter_by_month(year, month)
    else:
        fig = scatter_by_month()
        
    return fig

In [9]:
app.run_server( host='127.0.0.1', port='8050')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [22/Jun/2023 14:33:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2023 14:33:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2023 14:33:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2023 14:33:50] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2023 14:33:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2023 14:33:51] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
WebBrowser('http://127.0.0.1:8050', width=width, height=height)